# Arreglos irregulares, desiguales y Awkward Array

## ¿Qué es Awkward Array?

La lección anterior incluía un corte complicado:

```python
corte = muones["nMuon"] == 2

pt0 = muones["Muon_pt", corte, 0]
```

Las tres partes de `muones["Muon_pt", corte, 0]` son:

1. selecciona el campo `"Muon_pt"` de todos los registros en la matriz,
2. aplica `corte`, una matriz booleana, para seleccionar solo los eventos con dos muones,
3. selecciona el primer (`0`) muón de cada uno de esos pares. De manera similar, para el segundo (`1`) muón.

NumPy no podría realizar un corte así, ni siquiera representar una matriz de listas de longitud variable sin recurrir a arreglos de objetos.

In [ ]:
import numpy as np

np.array([[0.0, 1.1, 2.2], [], [3.3, 4.4], [5.5], [6.6, 7.7, 8.8, 9.9]])

Awkward Array está diseñado para llenar este vacío:

In [ ]:
import awkward as ak

ak.Array([[0.0, 1.1, 2.2], [], [3.3, 4.4], [5.5], [6.6, 7.7, 8.8, 9.9]])

Arreglos como este se llaman "irregulares" o "desiguales" (en inglés, ["jagged arrays"](https://en.wikipedia.org/wiki/Jagged_array) o a veces "ragged arrays").

### Rebanadas en Awkward Array

Las rebanadas básicas son una generalización de los de NumPy. Lo que NumPy haría si tuviera listas de longitud variable.

In [ ]:
array = ak.Array([[0.0, 1.1, 2.2], [], [3.3, 4.4], [5.5], [6.6, 7.7, 8.8, 9.9]])
array.tolist()

In [ ]:
array[2]

In [ ]:
array[-1, 1]

In [ ]:
array[2:, 0]

In [ ]:
array[2:, 1:]

In [ ]:
array[:, 0]

**Quiz rápido:** ¿por qué el último genera un error?

Las rebanadas con booleanos y enteros también funcionan.

In [ ]:
array[[True, False, True, False, True]]

In [ ]:
array[[2, 3, 3, 1]]

Como en NumPy, se pueden calcular arreglos booleanos para rebanadas, y funciones como [ak.num](https://awkward-array.readthedocs.io/en/latest/_auto/ak.num.html) son útiles para eso.

In [ ]:
ak.num(array)

In [ ]:
ak.num(array) > 0

In [ ]:
array[ak.num(array) > 0, 0]

In [ ]:
array[ak.num(array) > 1, 1]

Ahora considera esto (similar a un ejemplo de la primera lección):

In [ ]:
corte = array * 10 % 2 == 0

array[corte]

Este arreglo, `corte`, no es solo un arreglo de booleanos. Es un arreglo irrecular de booleanos. Todas sus listas anidadas encajan en las listas anidadas de `array`, por lo que puede seleccionar profundamente números, en lugar de seleccionar listas.

### Aplicación: seleccionando partículas, en lugar de eventos

Volviendo al TTree grande de la lección anterior,

In [ ]:
import uproot

archivo = uproot.open(
    "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
)
tree = archivo["Events"]

muon_pt = tree["Muon_pt"].array(entry_stop=10)
muon_pt

Este arreglo irregular de booleanos selecciona todos los *muones* con al menos 20 GeV:

In [ ]:
corte_particula = muon_pt > 20

muon_pt[corte_particula]

y este arreglo de booleanos no irregular (hecho con [ak.any](https://awkward-array.readthedocs.io/en/latest/_auto/ak.any.html)) selecciona todos los eventos *que tienen* un muón con al menos 20 GeV:

In [ ]:
corte_evento = ak.any(muon_pt > 20, axis=1)

muon_pt[corte_evento]

**Quiz rápido:** construye exactamente el mismo `corte_evento` utilizando [ak.max](https://awkward-array.readthedocs.io/en/latest/_auto/ak.max.html).

**Quiz rápido:** aplica ambos cortes; es decir, selecciona muones con más de 20 GeV de los eventos que los tienen.

Sugerencia: querrás hacer un

```python
muones_seleccionados = muon_pt[corte_particula]
```
intermediario y no puedes usar la variable `corte_evento` tal como está.

Sugerencia: el resultado final debería ser un arreglo irregular, al igual que `muon_pt`, pero con menos listas y menos elementos en esas listas.

````{admonition} Solución
:class: dropdown
```python
muones_seleccionados = muon_pt[corte_particula]
resultado_final = muones_seleccionados[corte_evento]
resultado_final.tolist()
```
````

## Combinatoria en Awkward Array

Las listas de longitud variable presentan más problemas que solo el corte y el cálculo de fórmulas en arrays. A menudo, queremos combinar partículas en todos los pares posibles (dentro de cada evento) para buscar cadenas de descomposición.

### Pares de dos arrays, pares de un solo array

Awkward Array tiene funciones que generan estas combinaciones. Por ejemplo, [ak.cartesian](https://awkward-array.readthedocs.io/en/latest/_auto/ak.cartesian.html) toma un producto cartesiano por evento (cuando `axis=1`, el valor predeterminado).

![cartoon-cartesian](fig/cartoon-cartesian.png)

In [ ]:
numeros = ak.Array([[1, 2, 3], [], [5, 7], [11]])
letras = ak.Array([["a", "b"], ["c"], ["d"], ["e", "f"]])

pares = ak.cartesian((numeros, letras))
pares

Estos `pares` son 2-tuplas, que son como registros en cómo se extraen de un array: usando cadenas.

In [ ]:
pares["0"]

In [ ]:
pares["1"]

También hay [ak.unzip](https://awkward-array.readthedocs.io/en/latest/_auto/ak.unzip.html), que extrae cada campo en un array separado (lo opuesto de [ak.zip](https://awkward-array.readthedocs.io/en/latest/_auto/ak.zip.html)).

In [ ]:
izquierda, derecha = ak.unzip(pares)
izquierda

In [ ]:
derecha

Tenga en cuenta que estos `izquierda` y `derecha` no son los `numeros` y `letras` originales: han sido duplicados y tienen la misma forma.

El producto cartesiano es equivalente a este bucle `for` en C++ sobre dos colecciones:

```cpp
for (int i = 0; i < numeros.size(); i++) {
  for (int j = 0; j < letras.size(); j++) {
    // formula con numeros[i] y letras[j]
  }
}
```

A veces, sin embargo, queremos encontrar todos los pares dentro de una sola colección, sin repetición. Eso sería equivalente a este bucle `for` en C++:

```cpp
for (int i = 0; i < numeros.size(); i++) {
  for (int j = i + 1; i < numeros.size(); j++) {
    // formula con numeros[i] y numeros[j]
  }
}
```

La función Awkward para este caso es [ak.combinations](https://awkward-array.readthedocs.io/en/latest/_auto/ak.combinations.html).

![cartoon-combinations](fig/cartoon-combinations.png)

In [ ]:
pares = ak.combinations(numeros, 2)
pares

In [ ]:
izquierda, derecha = ak.unzip(pares)
izquierda * derecha  # Se alinean, por lo que podemos calcular fórmulas

### Aplicación a los dimuones

La búsqueda de dimuones en la lección anterior fue un poco ingenua en el sentido de que requeríamos *exactamente dos* muones en cada evento y solo calculamos la masa de esa combinación. Si hubiera un tercer muón presente debido a una compleja descomposición electrodébil o porque algo fue medido incorrectamente, estaríamos ciegos a los otros dos muones. Podrían ser dimuones reales.

Un mejor procedimiento sería buscar todos los pares de muones en un evento y aplicar algunos criterios para seleccionarlos.

En este ejemplo, juntaremos (usando [ak.zip](https://awkward-array.readthedocs.io/en/latest/_auto/ak.zip.html)) las variables de los muones en registros.

In [ ]:
import uproot
import awkward as ak

archivo = uproot.open(
    "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
)
tree = archivo["Events"]

arrays = tree.arrays(filter_name="/Muon_(pt|eta|phi|charge)/", entry_stop=10000)

muones = ak.zip(
    {
        "pt": arrays["Muon_pt"],
        "eta": arrays["Muon_eta"],
        "phi": arrays["Muon_phi"],
        "charge": arrays["Muon_charge"],
    }
)

print(arrays.type)
print(muones.type)

La diferencia entre `arrays` y `muones` es que `arrays` contiene listas separadas de `"Muon_pt"`, `"Muon_eta"`, `"Muon_phi"`, `"Muon_charge"`, mientras que `muones` contiene listas de registros con los campos `"pt"`, `"eta"`, `"phi"`, `"charge"`.

Ahora podemos calcular pares de *objetos* muones.

In [ ]:
pares = ak.combinations(muones, 2)
pares.type

y separarlos en arreglos del primer muón y del segundo muón en cada par.

In [ ]:
mu1, mu2 = ak.unzip(pares)

**Quiz rápido:** ¿cómo garantizarías que todas las listas de registros en `mu1` y `mu2` tengan las mismas longitudes? Sugerencia: consulta [ak.num](https://awkward-array.readthedocs.io/en/latest/_auto/ak.num.html) y [ak.all](https://awkward-array.readthedocs.io/en/latest/_auto/ak.all.html).

Dado que tienen las mismas longitudes, podemos usarlos en una fórmula.

In [ ]:
import numpy as np

masa = np.sqrt(
    2 * mu1.pt * mu2.pt * (np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))
)

**Quiz rápido:** ¿cuántas masas tenemos en cada evento? ¿Cómo se compara esto con `muons`, `mu1` y `mu2`?

### Graficando el arreglo irregular

Dado que esta `masa` es un arreglo irregular, no se puede histogramar directamente. Los histogramas toman un conjunto de *números* como entradas, pero este arreglo contiene *listas*.

Suponiendo que solo deseas graficar los números de las listas, puedes usar [ak.flatten](https://awkward-array.readthedocs.io/en/latest/_auto/ak.flatten.html) para aplanar un nivel de lista o [ak.ravel](https://awkward-array.readthedocs.io/en/latest/_auto/ak.ravel.html) para aplanar todos los niveles.

In [ ]:
import hist

hist.Hist(hist.axis.Regular(120, 0, 120, label="masa [GeV]")).fill(
    ak.ravel(masa)
).plot()

Alternativamente, supongamos que deseas graficar la *máxima* masa candidata en cada evento, sesgándola hacia los bosones Z. [ak.max](https://awkward-array.readthedocs.io/en/latest/_auto/ak.max.html) es una función diferente que selecciona un elemento de cada lista, cuando se utiliza con `axis=1`.

In [ ]:
ak.max(masa, axis=1)

Algunos valores son `None` porque no hay un máximo en una lista vacía. [ak.flatten](https://awkward-array.readthedocs.io/en/latest/_auto/ak.flatten.html)/[ak.ravel](https://awkward-array.readthedocs.io/en/latest/_auto/ak.ravel.html) eliminan los valores faltantes (`None`) así como aplastan las listas,

In [ ]:
ak.flatten(ak.max(masa, axis=1), axis=0)

pero también lo hace eliminar las listas vacías en primer lugar.

In [ ]:
ak.max(masa[ak.num(masa) > 0], axis=1)

`````{admonition} Ejercicio: seleccionar pares de muones con cargas opuestas
Este no es un corte a nivel de evento ni un corte a nivel de partículas, es un corte sobre *pares* de partículas.

````{toggle} Solución
Las variables `mu1` y `mu2` son las mitades izquierda y derecha de los pares de muones. Por lo tanto,

```python
corte = (mu1.charge != mu2.charge)
```
tiene la multiplicidad correcta para aplicarse a la matriz `masa`.

```python
hist.Hist(hist.axis.Regular(120, 0, 120, label="masa [GeV]")).fill(
    ak.ravel(mass[cut])
).plot()
```
plotea los pares de muones seleccionados.
````
`````

`````{admonition} Ejercicio (más difícil): traza el candidato a masa por evento que esté estrictamente más cercano a la masa del Z
En lugar de solo tomar la masa máxima en cada evento, encuentra la que tenga la diferencia mínima entre la masa calculada y `masa_z = 91`.

Sugerencia: usa [ak.argmin](https://awkward-array.readthedocs.io/en/latest/_auto/ak.argmin.html) con `keepdims=True`.

Anticipando una de las futuras lecciones, podrías obtener una masa más precisa pidiendo a la librería Particle:

```python
import particle, hepunits

masa_z = particle.Particle.findall("Z0")[0].mass / hepunits.GeV
```

````{toggle} Solución
En lugar de maximizar `masa`, queremos minimizar `abs(masa - masa_z)` y aplicar esa elección a `masa`. [ak.argmin](https://awkward-array.readthedocs.io/en/latest/_auto/ak.argmin.html) devuelve la *posición del índice* de esta diferencia mínima, que luego podemos aplicar a la `masa` original. Sin embargo, sin `keepdims=True`, [ak.argmin](https://awkward-array.readthedocs.io/en/latest/_auto/ak.argmin.html) elimina la dimensión que necesitaríamos para que esta matriz tenga la misma forma anidada que `masa`. Por lo tanto, usamos `keepdims=True` y luego utilizamos [ak.ravel](https://awkward-array.readthedocs.io/en/latest/_auto/ak.ravel.html) para deshacernos de los valores faltantes y aplanar las listas.

El último paso requeriría dos aplicaciones de [ak.flatten](https://awkward-array.readthedocs.io/en/latest/_auto/ak.flatten.html): una para aplastar listas en el primer nivel y otra para eliminar `None` en el segundo nivel.

```python
cual = ak.argmin(abs(masa - masa_z), axis=1, keepdims=True)

hist.Hist(hist.axis.Regular(120, 0, 120, label="masa [GeV]")).fill(
    ak.ravel(masa[cual])

).plot()
```
````
`````